### Notes on the various typical figures of merit for spacecraft thrusters

Samuel Badman 02/19/19 samuel_badman@berkeley.edu

### Intrinsic Properties


#### Mass Flow Rate \[kg/s\]

The rate at which the thruster expels mass from the spacecraft. This is also intrinsic to the thruster. Denote as $\dot{m}$

#### Wet Mass / Dry Mass \[kg\]

For dynamical calcuations, these quantites include the total mass of the spacecraft. The wet mass is the total mass of the spacecraft when the thruster is fully fueled. The dry mass is the mass of the spacecraft with 0 propellent. Denote as $M_0$ and $M_f$.

#### Exhaust Velocity \[m/s\]

The velocity with which propellent is expelled by the thruster in the reference frame moving with the thruster. Denote as $v_e$.

#### Specific Impulse \[s\]

Another intrinsic property of the thruster given in units of seconds. It is simply a rescaling of the exhaust velocity via standard gravity $g_0=9.81$ms$^{-2}$ such that :

$$
I_{SP} = v_e/g_0
$$

#### Thrust \[N\]

The instantaenous amount of force applied to the spacecraft center of mass by a thruster firing. It is related to the mass flow rate ($\dot{m}$) and the specific impulse as :

$$
F = g_0 I_{sp} \dot{m} = v_e \dot{m}
$$


### Derived Quantities

#### Delta v

The total change in velocity in the direction of a burn in a fixed direction as a function of time is found as follows :

The inertial acceleration of a rocket of mass $m(t)$ with exhaust velocity $v_e$ and mass flow rate $\dot{m}$ is :

$$
m\frac{dv}{dt} = -v_e \dot{m}
$$

This is the [Tsiolkovsky Rocket Equation](https://en.wikipedia.org/wiki/Tsiolkovsky_rocket_equation)

Which has the solution for fixed $v_e$ for the change in velocity as :

$$
\Delta v(m) = -v_e \ln \frac{m(t)}{m_0}
$$

Assuming a constant mass flow rate, we have $m(t) = m_0 - \dot{m}t$, so as a function of time :

$$
\Delta v(t) = -v_e \ln (1 - \frac{\dot{m}}{m_0}t)
$$

We can also get the total delta v budget for a given thruster by letting $m(t) \to m_f$, the spacecraft dry mass.

$$
\Delta v_{tot} = -v_e \ln \frac{m_f}{m_0}
$$


#### Burn Time

We can invert the equation for $\Delta v (t)$ to get the time required for a given speed change $\Delta t$ :

$$
t_{burn} = \frac{m_0}{\dot{m}}(1-\exp{\bigg(-\frac{\Delta v}{v_e}}\bigg))
$$

Which can also be expressed in terms of the thrust as :

$$
t_{burn} = \frac{m_0v_e}{F}(1-\exp{\bigg(-\frac{\Delta v}{v_e}}\bigg))
$$

We see a useful linear approximation that if $\Delta v \ll v_e$ then :

$$
t_{burn} = \frac{m_0\Delta v}{F}
$$

The maximum time that the thruster can operate for is also useful and is straightforward to find as :

$$
m(t=t_{max}) = m_0 - \dot{m}t_{max} = m_f \\
\implies t_{max} = \frac{m_0 - m_f}{\dot{m}} = \frac{m_p}{\dot{m}} $$
where we define the total mass of propellent, $m_p$.

#### Total Impulse

The maximum change in momentum that the thruster can deliver to the spacecraft is found by integrating the thrust over the time that thrust can be applied for. Since the thrust is constant, this is simply a multiplication factor :

$$ I_T = Ft_{max} = \frac{m_pF}{\dot{m}} = v_e m_p $$

To relate this to the total Delta v budget, we have that :

$$
\Delta v_{tot} = -v_e \ln \frac{m_f}{m_0} = -v_e \ln (1 - \frac{m_p}{m_0})
$$

We see that in the limit where the propellent is a small fraction of the wet mass, then :

$$
\Delta v_{tot} \approx v_e \frac{m_p}{m_0} = \frac{I_T}{m_0}
$$

i.e. in this limit the total delta v budget is approximately the total impulse divided by the spacecraft wet mass. 

The 2nd order correction gives :

$$
\Delta v_{tot} \approx -v_e (-\frac{m_p}{m_0} - \frac{1}{2}\frac{m_p^2}{m_0^2}) = \frac{I_T}{m_0}(1 + \frac{m_p}{2m_0})
$$

i.e. for a given quoted total impulse, the 1st order approximation of $\Delta v$ is an _underestimate_ and so the total impulse over the spacecraft wet mass places a conservative lower bound on the spacecraft $\Delta v$ budget.

### Example: VACCO Standard MiPS for 6kg cubesat

[Spec Sheet](https://www.vacco.com/images/uploads/pdfs/MiPS_standard_0714.pdf) gives :

$F = 10$ mN, $I_{SP} = 40$ s, $I_T = 44$ Ns

In [1]:
# In SI Units
F = 10e-3 #N
I_SP = 40 #s
I_T = 44 #Ns
g_0 = 9.81 #m/s^2

We will assume the spacecraft wet mass is $6$kg

In [2]:
m_0 = 6.0 #kg

We can immediately get the exhaust velocity :

In [3]:
v_e = I_SP*g_0
print(f"Exhaust velocity = {v_e:0.8f} m/s")

Exhaust velocity = 392.40000000 m/s


And mass flow rate :

In [4]:
m_dot = F/v_e
print(f"Mass Flow Rate = {m_dot:0.8f} kg/s")

Mass Flow Rate = 0.00002548 kg/s


Note we can already see that approximations like $\Delta v \ll v_e$ will be strongly valid.

Of the intrinsic quantities, we haven't directly been given the spacecraft dry mass or equivalently, the propellent mass, but we can back this out of the total impulse :

In [5]:
m_p = I_T/v_e 
print(f"Propellent Mass = {m_p:0.8f} kg")

Propellent Mass = 0.11213048 kg


We can see that the quantity $m_p/m_0$ is small:

In [6]:
print(f'Propellent mass : wet mass ratio = {m_p/m_0:0.5f}')

Propellent mass : wet mass ratio = 0.01869


So a crude estimate of the total delta v budget should be accurate, and an underestimate by ~0.9% :

In [7]:
delta_v_est = I_T/m_0
print(f'Delta V Crude Estimate = {delta_v_est:2.3f} m/s')

Delta V Crude Estimate = 7.333 m/s


A more accurate estimate can be obtained as described above

In [8]:
import numpy as np
delta_v_accurate = -v_e*np.log(1-m_p/m_0)
print(f'Delta V Accurate Estimate = {delta_v_accurate:2.3f} m/s')

Delta V Accurate Estimate = 7.403 m/s


And checking the error:

In [10]:
print(f'Estimate Error : {(delta_v_accurate - delta_v_est)/delta_v_est*100 :2.5f} %')

Estimate Error : 0.94623 %


Which is what we expected.